In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
subm = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
subm.head()

In [ ]:
train.head()

In [ ]:
temp=train.copy()


for j,i in enumerate(train.target):
    if i==1:
        temp.set_value(j,'positive',1)
        temp.set_value(j,'negative',0)
    else:
        temp.set_value(j,'positive',0)
        temp.set_value(j,'negative',1)
        
        
temp['positive'].fillna(0,inplace=True)
temp['negative'].fillna(1,inplace=True)
temp.head(30)
train=temp.copy()
train.head()

In [ ]:
COMMENT = 'text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [ ]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [ ]:
trn_term_doc, test_term_doc

In [ ]:
x = trn_term_doc
test_x = test_term_doc
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)


In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [ ]:
label_cols=['positive','negative']

In [ ]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

In [ ]:
target=[] 3 to get all the target values of classifier

In [ ]:
submid = pd.DataFrame({'id': subm["id"]})
for i in range(len(preds)):
    if preds[i][0]>preds[i][1]:
        target.append(1)
    else:
        target.append(0)


In [ ]:
submid['target']=target

submid.to_csv('submission.csv', index=False)
submid.head(10)

# Mini Example Below to understand whole process

# Training Data

In [2]:
checker=pd.DataFrame({'Text':['He is playing positively',
                             'He is playing negatively',
                             'He playing positively'],
                     'positive':[1,0,1],
                     'negative':[0,1,0]})
checker.head()

,Text,positive,negative
0,He is playing positively,1,0
1,He is playing negatively,0,1
2,He playing positively,1,0


# Test Data

In [3]:
checker_test=pd.DataFrame({'Text':['He is playing positively',
                             'she is  negatively'],
                     })

In [4]:
vec = TfidfVectorizer(ngram_range=(1,2),
               min_df=1, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
tokenised_text=vec.fit_transform(checker['Text'])
print(vec.get_feature_names())

test_data=vec.transform(checker_test['Text'])

['he is', 'he playing', 'is', 'is playing', 'negatively', 'playing negatively', 'playing positively', 'positively']


In [5]:
print(tokenised_text.shape)
print('\n')
print(tokenised_text.todense()[0])
print('\n')
print(tokenised_text.todense()[2])


(3, 8)


[[0.4472136 0.        0.4472136 0.4472136 0.        0.        0.4472136
  0.4472136]]


[[0.         0.68091856 0.         0.         0.         0.
  0.51785612 0.51785612]]


In [13]:
print(x.todense())

[[0.4472136  0.         0.4472136  0.4472136  0.         0.
  0.4472136  0.4472136 ]
 [0.3935112  0.         0.3935112  0.3935112  0.51741994 0.51741994
  0.         0.        ]
 [0.         0.68091856 0.         0.         0.         0.
  0.51785612 0.51785612]]


In [6]:
x=tokenised_text
def pr(y_i, y):
    print(y_i==y)
    print('\n')
    p = x[y==y_i].sum(0)
    print(p)
    return (p+1) / ((y==y_i).sum()+1)


In [20]:
def get_mdl(y):
    y = y.values
    print('Values are',y)
    r = np.log(pr(1,y) / pr(0,y))
    print(r,"\n")
    m = LogisticRegression(C=4)
    x_nb = x.multiply(r)
    print("We are ",x_nb.todense())
    return m.fit(x_nb, y), r

In [21]:
label_cols=['positive','negative']
preds = np.zeros((len(checker_test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(checker[j])
    preds[:,i] = m.predict_proba(test_data.multiply(r))[:,1]

fit positive
Values are [1 0 1]
[ True False  True]


[[0.4472136  0.68091856 0.4472136  0.4472136  0.         0.
  0.96506971 0.96506971]]
[False  True False]


[[0.3935112  0.         0.3935112  0.3935112  0.51741994 0.51741994
  0.         0.        ]]
[[-0.36765167  0.1138753  -0.36765167 -0.36765167 -0.8224766  -0.8224766
   0.27006261  0.27006261]] 

We are  [[-0.16441882  0.         -0.16441882 -0.16441882  0.          0.
   0.12077567  0.12077567]
 [-0.14467505  0.         -0.14467505 -0.14467505 -0.42556579 -0.42556579
   0.          0.        ]
 [ 0.          0.0775398   0.          0.          0.          0.
   0.13985358  0.13985358]]
fit negative
Values are [0 1 0]
[False  True False]


[[0.3935112  0.         0.3935112  0.3935112  0.51741994 0.51741994
  0.         0.        ]]
[ True False  True]


[[0.4472136  0.68091856 0.4472136  0.4472136  0.         0.
  0.96506971 0.96506971]]
[[ 0.36765167 -0.1138753   0.36765167  0.36765167  0.8224766   0.8224766
  -0.27006261 -0